In [ ]:
!pip install yfinance

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [ ]:
def calculate_volatility(symbol, per):
    # historical data from Yahoo Finance
    stock_data = symbol.history(period = per)

    stock_data['Returns'] = stock_data['Close'].pct_change()

    # volatility -> standard deviation
    volatility = np.std(stock_data['Returns'])

    return volatility

In [ ]:
def calculate_bollinger_bands(symbol, per, window=20, num_std=2):

    stock_data = symbol.history(period=per)

    stock_data['Returns'] = stock_data['Close'].pct_change()

    #also get volatility
    stock_data['Volatility'] = stock_data['Returns'].rolling(window=window).std()

    # calculate the rolling mean and standard deviation for Bollinger Bands
    stock_data['SMA'] = stock_data['Close'].rolling(window=window).mean()
    stock_data['Upper Band'] = stock_data['SMA'] + (num_std * stock_data['Close'].rolling(window=window).std())
    stock_data['Lower Band'] = stock_data['SMA'] - (num_std * stock_data['Close'].rolling(window=window).std())

    return stock_data[['Close', 'SMA', 'Upper Band', 'Lower Band', 'Volatility']]

#returns a dataframe of things

In [ ]:
def macd_calc(symbol, per):
    stock_data = symbol.history(period=per)
    stock_data.get('Volume')
    stock_data.get('Close')

    stock_data['EMA12'] = stock_data['Close'].ewm(span=12, min_periods=0, adjust=False).mean()
    stock_data['EMA26'] = stock_data['Close'].ewm(span=26, min_periods=0, adjust=False).mean()
    stock_data['MACD'] = stock_data['EMA12'] - stock_data['EMA26']
    stock_data['Signal'] = stock_data['MACD'].ewm(span=9, min_periods=0, adjust=False).mean()
    stock_data['Histogram'] = stock_data['MACD'] - stock_data['Signal']

    return stock_data[['EMA12', 'EMA26', 'MACD', 'Signal', 'Histogram']]

In [ ]:
def sma_calc(symbol, per):
    stock_data = symbol.history(period=per)
    stock_data['SMA50'] = stock_data['Close'].rolling(50).mean()
    stock_data['SMA200'] = stock_data['Close'].rolling(200).mean()

    return stock_data [['SMA50', 'SMA200']]


In [ ]:
def rsi_cal(symbol, per):
    stock_data = symbol.history(period=per)
    stock_data['RSI'] = 100 - 100 / (
            1 + (stock_data['Close'].diff() / stock_data['Close'].shift(1)).rolling(14).mean())

    return stock_data ['RSI']

In [ ]:
symbol = yf.Ticker('YUM')
per = '10y'

volatility = calculate_volatility(symbol, per)
bollinger_bands_data = calculate_bollinger_bands(symbol, per)
macd = macd_calc(symbol, per)
sma = sma_calc(symbol, per)
rsi = rsi_cal(symbol, per)


stock_data_df = bollinger_bands_data.join([macd, sma, rsi])



In [ ]:
stock_data_df = stock_data_df.dropna()

In [ ]:
stock_data_df

,Close,SMA,Upper Band,Lower Band,Volatility,EMA12,EMA26,MACD,Signal,Histogram,SMA50,SMA200,RSI
Date,,,,,,,,,,,,,
2014-11-07 00:00:00-05:00,44.201542,42.415561,44.686598,40.144525,0.010420,43.299822,42.923108,0.376714,0.082054,0.294660,42.921366,44.682383,0.378645
2014-11-10 00:00:00-05:00,44.450275,42.599343,44.905881,40.292806,0.008219,43.476815,43.036232,0.440583,0.153760,0.286823,42.936871,44.702005,0.349824
2014-11-11 00:00:00-05:00,44.601944,42.784680,45.110277,40.459083,0.008215,43.649912,43.152210,0.497701,0.222548,0.275153,42.967590,44.721310,0.442024
2014-11-12 00:00:00-05:00,45.232876,43.007023,45.377907,40.636139,0.008316,43.893445,43.306334,0.587111,0.295461,0.291650,43.010203,44.748817,0.529809
2014-11-13 00:00:00-05:00,45.736401,43.242711,45.717889,40.767534,0.008420,44.176976,43.486339,0.690638,0.374496,0.316141,43.060476,44.778573,0.544183
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-19 00:00:00-05:00,132.300003,129.615000,131.638979,127.591022,0.006407,129.806262,129.093202,0.713060,0.625674,0.087387,127.941194,129.588704,0.098883
2024-01-22 00:00:00-05:00,131.059998,129.716000,131.819215,127.612785,0.006420,129.999145,129.238891,0.760254,0.652590,0.107664,128.064609,129.595183,0.024366
2024-01-23 00:00:00-05:00,131.750000,129.804501,132.095099,127.513902,0.006328,130.268507,129.424899,0.843608,0.690794,0.152815,128.199236,129.603436,0.150858
